In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime

In [2]:
# Stupid but smart
def getCorrespondingStartDate(date):
    month = date.month
    year = date.year
    if month - 10 >= 0: 
        return datetime.date(year, 10, 1)
    if month - 7 >= 0:
        return datetime.date(year, 7, 1)
    if month - 4 >= 0:
        return datetime.date(year, 4, 1)
    return datetime.date(year, 1, 1)

In [25]:
# here's my plan. i iterate through features separately. so i can sort by $exists = True.
# helps avoid those annoying key errors, and the mathematical badness of setting nan = 0
# when you're trying to calculate an average. 
# it's NOT efficient, but we're only doing it once. right??
def getDataAnomQuarterly(start_date, end_date):
    client = pymongo.MongoClient('localhost', 27017, maxPoolSize=100)
    db = client['Stocks']
    stocks = ["AAPL US Equity", "WWW US Equity"]
    curr_stock = db["AAPL US Equity"]

    fields = ['LAST_PRICE', 'EBITDA', 'HISTORICAL_MARKET_CAP']
    
    date_index = pd.date_range(getCorrespondingStartDate(start_date),
                        getCorrespondingStartDate(end_date), 
                       freq='QS')
    
    s_len = len(stocks)
    f_len = len(fields)
    
    a = np.empty((s_len, f_len))
    a[:]= np.nan
    
    df = pd.DataFrame({"Array": [a]}, index=date_index)

    
    for i in range(s_len):
        
        curr_date = None
        curr_total = 0
        curr_count = 0
    
        for j in range(f_len): 
            field = fields[j]
            
            # TO DO sort by date too!!!!
            for thing in curr_stock.find({field:{'$exists': True}}).sort([('Date', 1)]):
                date = thing['Date']
                corr_date = getCorrespondingStartDate(date)

                if curr_date is None:
                    curr_count += 1
                    curr_date = corr_date
                    curr_total = thing[field]
                elif (corr_date == curr_date):
                    curr_count += 1
                    curr_total += thing[field]   
                # If the dates don't match, we save the info before moving on. 
                else:
                    arr = df.loc[curr_date]

                    arr[0][i][j] = curr_total/curr_count
                    
                    df.loc[curr_date] = arr
                    
                    curr_date = corr_date
                    curr_count = 1
                    curr_total = thing[field]

    return df

In [26]:
start_date = datetime.date(1990, 1, 1)
end_date = datetime.date(2019, 1, 1)

dataDict = getDataAnomQuarterly(start_date, end_date)

0
0
Array    [[1.2887, nan, nan], [nan, nan, nan]]
Name: 1990-01-01 00:00:00, dtype: object
Array    [[1.2887, nan, nan], [nan, nan, nan]]
Name: 1990-01-01 00:00:00, dtype: object
0
0
Array    [[1.4925666666666668, nan, nan], [nan, nan, nan]]
Name: 1990-04-01 00:00:00, dtype: object
Array    [[1.4925666666666668, nan, nan], [nan, nan, nan]]
Name: 1990-04-01 00:00:00, dtype: object
0
0
Array    [[1.2857, nan, nan], [nan, nan, nan]]
Name: 1990-07-01 00:00:00, dtype: object
Array    [[1.2857, nan, nan], [nan, nan, nan]]
Name: 1990-07-01 00:00:00, dtype: object
0
0
Array    [[1.3154666666666668, nan, nan], [nan, nan, nan]]
Name: 1990-10-01 00:00:00, dtype: object
Array    [[1.3154666666666668, nan, nan], [nan, nan, nan]]
Name: 1990-10-01 00:00:00, dtype: object
0
0
Array    [[2.1517666666666666, nan, nan], [nan, nan, nan]]
Name: 1991-01-01 00:00:00, dtype: object
Array    [[2.1517666666666666, nan, nan], [nan, nan, nan]]
Name: 1991-01-01 00:00:00, dtype: object
0
0
Array    [[1.70833333333

Name: 2013-10-01 00:00:00, dtype: object
0
0
Array    [[74.45616666666666, nan, nan], [nan, nan, nan]]
Name: 2014-01-01 00:00:00, dtype: object
Array    [[74.45616666666666, nan, nan], [nan, nan, nan]]
Name: 2014-01-01 00:00:00, dtype: object
0
0
Array    [[89.21906666666666, nan, nan], [nan, nan, nan]]
Name: 2014-04-01 00:00:00, dtype: object
Array    [[89.21906666666666, nan, nan], [nan, nan, nan]]
Name: 2014-04-01 00:00:00, dtype: object
0
0
Array    [[99.61666666666667, nan, nan], [nan, nan, nan]]
Name: 2014-07-01 00:00:00, dtype: object
Array    [[99.61666666666667, nan, nan], [nan, nan, nan]]
Name: 2014-07-01 00:00:00, dtype: object
0
0
Array    [[112.43666666666667, nan, nan], [nan, nan, nan]]
Name: 2014-10-01 00:00:00, dtype: object
Array    [[112.43666666666667, nan, nan], [nan, nan, nan]]
Name: 2014-10-01 00:00:00, dtype: object
0
0
Array    [[123.35000000000001, nan, nan], [nan, nan, nan]]
Name: 2015-01-01 00:00:00, dtype: object
Array    [[123.35000000000001, nan, nan], [na

Name: 2004-07-01 00:00:00, dtype: object
0
1
Array    [[179.08, 444.0, nan], [nan, nan, nan]]
Name: 2004-10-01 00:00:00, dtype: object
Array    [[179.08, 444.0, nan], [nan, nan, nan]]
Name: 2004-10-01 00:00:00, dtype: object
0
1
Array    [[179.08, 484.0, nan], [nan, nan, nan]]
Name: 2005-01-01 00:00:00, dtype: object
Array    [[179.08, 484.0, nan], [nan, nan, nan]]
Name: 2005-01-01 00:00:00, dtype: object
0
1
Array    [[179.08, 472.0, nan], [nan, nan, nan]]
Name: 2005-04-01 00:00:00, dtype: object
Array    [[179.08, 472.0, nan], [nan, nan, nan]]
Name: 2005-04-01 00:00:00, dtype: object
0
1
Array    [[179.08, 463.0, nan], [nan, nan, nan]]
Name: 2005-07-01 00:00:00, dtype: object
Array    [[179.08, 463.0, nan], [nan, nan, nan]]
Name: 2005-07-01 00:00:00, dtype: object
0
1
Array    [[179.08, 802.0, nan], [nan, nan, nan]]
Name: 2005-10-01 00:00:00, dtype: object
Array    [[179.08, 802.0, nan], [nan, nan, nan]]
Name: 2005-10-01 00:00:00, dtype: object
0
1
Array    [[179.08, 631.0, nan], [na

Name: 1996-10-01 00:00:00, dtype: object
0
2
Array    [[179.08, 18633.0, 2354.6657], [nan, nan, nan]]
Name: 1997-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 2354.6657], [nan, nan, nan]]
Name: 1997-01-01 00:00:00, dtype: object
0
2
Array    [[179.08, 18633.0, 1858.8353], [nan, nan, nan]]
Name: 1997-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 1858.8353], [nan, nan, nan]]
Name: 1997-04-01 00:00:00, dtype: object
0
2
Array    [[179.08, 18633.0, 2726.913], [nan, nan, nan]]
Name: 1997-07-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 2726.913], [nan, nan, nan]]
Name: 1997-07-01 00:00:00, dtype: object
0
2
Array    [[179.08, 18633.0, 1704.2529], [nan, nan, nan]]
Name: 1997-10-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 1704.2529], [nan, nan, nan]]
Name: 1997-10-01 00:00:00, dtype: object
0
2
Array    [[179.08, 18633.0, 3582.445], [nan, nan, nan]]
Name: 1998-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 3582.445], [nan, nan, nan]]
Name

2
Array    [[179.08, 18633.0, 829383.854], [nan, nan, nan]]
Name: 2018-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [nan, nan, nan]]
Name: 2018-01-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [1.2887, nan, ...
Name: 1990-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [1.2887, nan, ...
Name: 1990-01-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [1.49256666666...
Name: 1990-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [1.49256666666...
Name: 1990-04-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [1.2857, nan, ...
Name: 1990-07-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [1.2857, nan, ...
Name: 1990-07-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [1.31546666666...
Name: 1990-10-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [1.31546666666...
Name: 1990-10-01 00:00:00

Name: 2016-01-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [96.3999999999...
Name: 2016-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [96.3999999999...
Name: 2016-04-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [107.786666666...
Name: 2016-07-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [107.786666666...
Name: 2016-07-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [113.293333333...
Name: 2016-10-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [113.293333333...
Name: 2016-10-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [134.0, nan, n...
Name: 2017-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [134.0, nan, n...
Name: 2017-01-01 00:00:00, dtype: object
1
0
Array    [[179.08, 18633.0, 829383.854], [146.809999999...
Name: 2017-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854]

1
Array    [[179.08, 18633.0, 829383.854], [179.08, 48.0,...
Name: 2001-10-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 48.0,...
Name: 2001-10-01 00:00:00, dtype: object
1
1
Array    [[179.08, 18633.0, 829383.854], [179.08, 57.0,...
Name: 2002-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 57.0,...
Name: 2002-01-01 00:00:00, dtype: object
1
1
Array    [[179.08, 18633.0, 829383.854], [179.08, 41.0,...
Name: 2002-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 41.0,...
Name: 2002-04-01 00:00:00, dtype: object
1
1
Array    [[179.08, 18633.0, 829383.854], [179.08, 18.0,...
Name: 2002-07-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 18.0,...
Name: 2002-07-01 00:00:00, dtype: object
1
1
Array    [[179.08, 18633.0, 829383.854], [179.08, 17.0,...
Name: 2002-10-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 17.0,...
Name: 2002-10-01 00:00:

Name: 1996-01-01 00:00:00, dtype: object
1
2
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1996-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1996-04-01 00:00:00, dtype: object
1
2
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1996-07-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1996-07-01 00:00:00, dtype: object
1
2
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1996-10-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1996-10-01 00:00:00, dtype: object
1
2
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1997-01-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1997-01-01 00:00:00, dtype: object
1
2
Array    [[179.08, 18633.0, 829383.854], [179.08, 18633...
Name: 1997-04-01 00:00:00, dtype: object
Array    [[179.08, 18633.0, 829383.854]

In [ ]:
print(dataDict)